<a href="https://colab.research.google.com/github/ktaran-jeet/3dVision/blob/DLNotes/Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##Linear module:
import torch
import torch.nn as nn

In [ ]:
#In the context of neural networks, a linear layer (also known as a fully connected layer or dense layer) applies a linear transformation to its input. This transformation is defined by the equation: y = x * W^T + b
module = nn.Linear(5,3)
print(module)

Linear(in_features=5, out_features=3, bias=True)


In [ ]:
ip = torch.rand(5)
print(ip)

tensor([0.5478, 0.0385, 0.7758, 0.4499, 0.9995])


In [ ]:
y = module(ip)
print(y)

tensor([-0.3043, -0.3537, -0.3266], grad_fn=<ViewBackward0>)


In [ ]:
print(module.weight)
print(module.weight.size())

Parameter containing:
tensor([[-0.0118,  0.3327, -0.2300, -0.3586, -0.0776],
        [ 0.0556,  0.0843, -0.2661, -0.2053, -0.1646],
        [-0.3621, -0.2757,  0.0608,  0.0439, -0.3471]], requires_grad=True)
torch.Size([3, 5])


In [ ]:
print(module.bias)
print(module.bias.size())

Parameter containing:
tensor([0.1067, 0.0759, 0.1624], requires_grad=True)
torch.Size([3])


In [ ]:
##The primary purpose of using with torch.no_grad(): is to temporarily disable gradient calculations within the code block it encloses.
## during inference, evaluation or durring manipulating model params directly.
with torch.no_grad():
  module.weight[0,0] = 0
  module.weight[2,0] = 0
  module.bias[0] = 0
  print(module.weight)
  print(module.bias)

Parameter containing:
tensor([[ 0.0000,  0.3327, -0.2300, -0.3586, -0.0776],
        [ 0.0556,  0.0843, -0.2661, -0.2053, -0.1646],
        [ 0.0000, -0.2757,  0.0608,  0.0439, -0.3471]], requires_grad=True)
Parameter containing:
tensor([0.0000, 0.0759, 0.1624], requires_grad=True)


In [2]:
## A linear module without bias:
mod = nn.Linear(2,3,bias=False)
print(mod)

Linear(in_features=2, out_features=3, bias=False)


In [3]:
print(mod.weight)
print(mod.bias)

Parameter containing:
tensor([[-0.4626,  0.2284],
        [ 0.0786,  0.1744],
        [-0.4646, -0.3355]], requires_grad=True)
None


In [20]:
x = torch.ones(2)
print(x)
y = mod(x)
print(y)

tensor([1., 1.])
tensor([ 3.,  7., 11.], grad_fn=<SqueezeBackward4>)


In [17]:
wm = torch.arange(1,7).reshape(3,2).float()
print(wm.dtype)
with torch.no_grad():
  mod.weight = nn.Parameter(wm)
  print(mod.weight)

torch.float32
Parameter containing:
tensor([[1., 2.],
        [3., 4.],
        [5., 6.]], requires_grad=True)


In [18]:
newy = mod(x)
print(newy)

tensor([ 3.,  7., 11.], grad_fn=<SqueezeBackward4>)


In [36]:
##Activation fucntion: Softmax:
import numpy as np
#numerical stability. When dealing with exponentials, values can become very large, leading to potential overflow errors. By subtracting the maximum value (np.max(x)) before exponentiation, we shift the values towards zero, reducing the risk of overflow. This ensures the function produces more reliable results, especially when dealing with large input values

def softmax(x):
  x = x-np.max(x)
  return np.exp(x)/np.sum(np.exp(x))

logits = np.array([1.0,2.0,3.0])
print(softmax(logits))
print(np.sum(softmax(logits)))

[0.09003057 0.24472847 0.66524096]
0.9999999999999999


In [44]:
import torch.nn.functional as F

logits = torch.tensor([1.0,2.0,3.0])
probs = F.softmax(logits, dim=0)
print(probs)
print(torch.sum(probs))

tensor([0.0900, 0.2447, 0.6652])
tensor(1.)


In [50]:
xx = torch.Tensor(torch.arange(1,13).reshape(3,4).float())
print(xx)
p = torch.nn.functional.softmax(xx,dim=0)
py = torch.nn.functional.softmax(xx,dim=1)
print(p)
print(p.sum(dim=0))
print('')
print(py)
print(py.sum(dim=1))

tensor([[ 1.,  2.,  3.,  4.],
        [ 5.,  6.,  7.,  8.],
        [ 9., 10., 11., 12.]])
tensor([[3.2932e-04, 3.2932e-04, 3.2932e-04, 3.2932e-04],
        [1.7980e-02, 1.7980e-02, 1.7980e-02, 1.7980e-02],
        [9.8169e-01, 9.8169e-01, 9.8169e-01, 9.8169e-01]])
tensor([1.0000, 1.0000, 1.0000, 1.0000])

tensor([[0.0321, 0.0871, 0.2369, 0.6439],
        [0.0321, 0.0871, 0.2369, 0.6439],
        [0.0321, 0.0871, 0.2369, 0.6439]])
tensor([1., 1., 1.])


In [51]:
## vanilla neural networks::
import torch.nn as nn

class simpleNet(nn.Module):

  def __init__(self, inputl, hiddenl, outputl):
    super(simpleNet, self).__init__()

    self.layer1 = nn.Linear(inputl, hiddenl, bias=True)
    self.layer2 = nn.Linear(hiddenl, outputl, bias=True)

  def forward(self,x):
    x = self.layer1(x)
    x = torch.relu(x)
    x = self.layer2(x)
    p = torch.softmax(x, dim=0)
    return p

snet = simpleNet(2,5,3)
print(snet)

simpleNet(
  (layer1): Linear(in_features=2, out_features=5, bias=True)
  (layer2): Linear(in_features=5, out_features=3, bias=True)
)


In [53]:
x = torch.Tensor([1,1])
print(x)
prob = snet.forward(x) ##both are equivalent!
prob2 = snet(x)
print(prob)
print(prob2)

tensor([1., 1.])
tensor([0.2062, 0.1620, 0.6318], grad_fn=<SoftmaxBackward0>)
tensor([0.2062, 0.1620, 0.6318], grad_fn=<SoftmaxBackward0>)


In [57]:
print(snet.layer1.weight)
print(snet.layer1.bias)

Parameter containing:
tensor([[ 0.1221,  0.3344],
        [-0.3274, -0.4230],
        [-0.2041, -0.3286],
        [ 0.0626, -0.4214],
        [ 0.0604,  0.0039]], requires_grad=True)
Parameter containing:
tensor([ 0.6126, -0.0934,  0.6503, -0.5885,  0.1455], requires_grad=True)


In [60]:
with torch.no_grad():
  snet.layer1.weight[0,1]=0
  snet.layer1.weight[2,0]=0
  print(snet.layer1.weight)

nprob = snet(x)
print(nprob)
## probablities changed with change in weights!

Parameter containing:
tensor([[ 0.1221,  0.0000],
        [-0.3274, -0.4230],
        [ 0.0000, -0.3286],
        [ 0.0626, -0.4214],
        [ 0.0604,  0.0039]], requires_grad=True)
tensor([0.2306, 0.1982, 0.5711], grad_fn=<SoftmaxBackward0>)


In [55]:
print(snet.layer2.weight)
print(snet.layer2.bias)

Parameter containing:
tensor([[-0.2065,  0.2463,  0.0035,  0.1825,  0.2718],
        [-0.3504,  0.1920,  0.2101,  0.1474, -0.2457],
        [ 0.2867, -0.2425, -0.2312, -0.1056, -0.0475]], requires_grad=True)
Parameter containing:
tensor([-0.2922, -0.2957,  0.3947], requires_grad=True)


In [56]:
list_of_param = list(snet.parameters() )
print(list_of_param)

[Parameter containing:
tensor([[ 0.1221,  0.3344],
        [-0.3274, -0.4230],
        [-0.2041, -0.3286],
        [ 0.0626, -0.4214],
        [ 0.0604,  0.0039]], requires_grad=True), Parameter containing:
tensor([ 0.6126, -0.0934,  0.6503, -0.5885,  0.1455], requires_grad=True), Parameter containing:
tensor([[-0.2065,  0.2463,  0.0035,  0.1825,  0.2718],
        [-0.3504,  0.1920,  0.2101,  0.1474, -0.2457],
        [ 0.2867, -0.2425, -0.2312, -0.1056, -0.0475]], requires_grad=True), Parameter containing:
tensor([-0.2922, -0.2957,  0.3947], requires_grad=True)]


In [63]:
### Training nn on mnist dataset:
import utils
from utils import check_mnist_dataset_exists
data_path=check_mnist_dataset_exists()

ModuleNotFoundError: No module named 'utils'